<a href="https://colab.research.google.com/github/fjadidi2001/DataScienceJourney/blob/master/PINNs_S4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import necessary libraries


In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# For handling differential equations
from scipy.integrate import odeint


# Generate the dataset


In [ ]:
# Generate synthetic data for the heat equation u_t = alpha * u_xx
alpha = 0.01  # Thermal diffusivity

# Define the domain
x = np.linspace(0, 1, 100)
t = np.linspace(0, 1, 100)
X, T = np.meshgrid(x, t)

# Define the initial condition u(x, 0) = sin(pi * x)
U0 = np.sin(np.pi * x)

# Define a function that represents the solution u(x, t)
def heat_eqn(U0, t, alpha):
    def model(U, t):
        Ux = np.gradient(U, x)
        Uxx = np.gradient(Ux, x)
        return alpha * Uxx
    return odeint(model, U0, t)

# Generate data
U = heat_eqn(U0, t, alpha)

# Flatten the meshgrid arrays for the PINN input
X_train = np.vstack([X.ravel(), T.ravel()]).T
U_train = U.ravel()


# Define the PINN Architecture


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Define the neural network model using TensorFlow/Keras
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(20, activation='tanh', input_shape=(2,)),
        tf.keras.layers.Dense(20, activation='tanh'),
        tf.keras.layers.Dense(20, activation='tanh'),
        tf.keras.layers.Dense(1, activation=None)
    ])
    return model

model = create_model()

# Define the Loss Function

In [6]:
def physics_informed_loss(model, X_train, U_train, alpha):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(X_train)
        u_pred = model(X_train)
        u_x = tape.gradient(u_pred, X_train[:, 0:1])
        u_t = tape.gradient(u_pred, X_train[:, 1:2])
    u_xx = tape.gradient(u_x, X_train[:, 0:1])
    del tape

    # Physics loss (PDE residual)
    physics_loss = tf.reduce_mean(tf.square(u_t - alpha * u_xx))

    # Data loss
    data_loss = tf.reduce_mean(tf.square(u_pred - U_train))

    # Total loss
    total_loss = data_loss + physics_loss
    return total_loss


#  Compile and Train the Model

In [7]:
optimizer = tf.keras.optimizers.Adam()

@tf.function
def train_step(X_train, U_train):
    with tf.GradientTape() as tape:
        loss = physics_informed_loss(model, X_train, U_train, alpha)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

# Training loop
epochs = 1000
for epoch in range(epochs):
    loss = train_step(X_train, U_train)
    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.numpy()}')


TypeError: in user code:

    File "<ipython-input-7-191499527ec4>", line 6, in train_step  *
        loss = physics_informed_loss(model, X_train, U_train, alpha)
    File "<ipython-input-6-a0ae8c4a7007>", line 7, in physics_informed_loss  *
        u_xx = tape.gradient(u_x, X_train[:, 0:1])

    TypeError: Argument `target` should be a list or nested structure of Tensors, Variables or CompositeTensors to be differentiated, but received None.
